In [1]:
import pandas as pd
import numpy as np
from MoviaBusDataset import MoviaBusDataset

In [2]:
train = MoviaBusDataset('../data/train', interpolation=True)
test = MoviaBusDataset('../data/test', interpolation=True)
train = pd.concat(train.dataframes).drop('TimeOfDay',axis=1).unstack().reset_index().rename(columns={0:"Speed"})
test = pd.concat(test.dataframes).drop('TimeOfDay',axis=1).unstack().reset_index().rename(columns={0:"Speed"})

In [3]:
time = pd.DatetimeIndex(train.Time)

#calculate the historical average
df_ha=train.groupby([time.hour,time.minute,'LinkRef']).mean()
#Rename the index
df_ha.index=df_ha.index.rename(['hour','minute','LinkRef'])
#reset the index
df_ha=df_ha.reset_index()
df_ha.head() 


,hour,minute,LinkRef,Speed
0,6,0,103524893:266482296:1195317079,NaN
1,6,0,103524902:1242123651:1195317083,NaN
2,6,0,103524902:1242123670:3396253208,NaN
3,6,0,103524902:3396253208:1242123651,NaN
4,6,0,108233932:576722:4279694303,NaN


In [4]:
#add a hour and minute column to the test dataframe
test['hour'] = test.Time.apply(lambda x: x.hour)
test['minute'] = test.Time.apply(lambda x: x.minute)

#merge the historical mean with the test test
test_merged = pd.merge(test, df_ha, on=['hour','minute','LinkRef'])[['Time','LinkRef','Speed_x','Speed_y']]
#rename columns
test_merged.columns=['Time','LinkRef','mean_truth','mean_prediction']
test_merged = test_merged[test_merged['mean_truth']!=0]
test_merged.head()

,Time,LinkRef,mean_truth,mean_prediction
0,2018-10-04 06:00:00,103524893:266482296:1195317079,NaN,NaN
1,2018-10-05 06:00:00,103524893:266482296:1195317079,NaN,NaN
2,2018-10-04 06:05:00,103524893:266482296:1195317079,8.414000,10.676667
3,2018-10-05 06:05:00,103524893:266482296:1195317079,NaN,10.676667
4,2018-10-04 06:10:00,103524893:266482296:1195317079,8.225846,11.282816


In [5]:
test.Time.apply(lambda x: x.time)

1.8424587905114789

In [6]:
MAE = abs(test_merged['mean_truth']-test_merged['mean_prediction']).mean()
MAE

2.032292638041423

In [6]:
test_merged2 = test_merged[test_merged['mean_truth']>10]
MAPE = abs((test_merged2['mean_truth']-test_merged2['mean_prediction'])/(test_merged2['mean_truth'])).mean()*100
MAPE

55.385216713850504

In [7]:
RMSE = np.sqrt(((test_merged['mean_truth']-test_merged['mean_prediction'])**2).mean())
RMSE

2.546347986857115